In [ ]:
import os
import tarfile
import re

# 源文件目录和输出目录
source_dir = "arxiv_papers/2406-20000-14000"  
output_dir = "extracted_tables/2406-20000-14000"  # 填写年月 

# 创建输出目录
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

def extract_tables(tex_file_content):
    # 去除 % 开头的注释行，要在表格提取前去除，不然注释的表格代码第一行的注释符号不会被提取到导致余留一行代码，所以下面的去除注释行移到前面
    tex_file_content = re.sub(r'%.*', '', tex_file_content)

    # 正则表达式匹配 \begin{table} 到 \end{table} 表格内容
    tables = re.findall(r'\\begin\{table\*?\}.*?\\end\{table\*?\}', tex_file_content, re.DOTALL)
    
    cleaned_tables = []
    for table in tables:

        # 检查是否为嵌套表格（包含 \begin{subtable} 或 \end{subtable}）或者 如果表格中包含多个 \begin{tabular}，跳过该表格
        if '\\begin{subtable}' in table or len(re.findall(r'\\begin\{tabular\}', table)) > 1:
            continue  # 跳过嵌套表格

        # 清理 \href 命令, 只保留命令中的文本部分
        table = re.sub(r'\\href\{[^}]*\}\{([^}]+)\}', r'\1', table)

        # 清理 \citep 命令
        table = re.sub(r'\\citep\{.*?\}', '', table)

        # 删除引用命令，如 \cite{}, \label{}, \ref{}
        table = re.sub(r'~?\\cite\{.*?\}', '', table)   # 删除 \cite{}
        table = re.sub(r'\\label\{.*?\}', '', table)  # 删除 \label{}
        table = re.sub(r'\\ref\{.*?\}', '', table)    # 删除 \ref{}

        # 删除 \caption 命令
        table = re.sub(r'\\caption\{([^{}]*|(?:[^{}]*\{[^{}]*\})*[^{}]*)\}', '', table)

        if '\\caption' in table:
            continue  # 跳过此表格


        # 去除 \cellcolor{...} 的颜色设置
        table = re.sub(r'\\cellcolor(\[.*?\])?\{.*?\}', '', table)

        # 仅保留 \textcolor 中的文本内容
        table = re.sub(r'\\textcolor\{.*?\}\{(.*?)\}', r'\1', table)  
        # 仅保留 \colorbox 中的文本内容
        table = re.sub(r'\\colorbox\{.*?\}\{(.*?)\}', r'\1', table)

        # 删除多余的换行和空行
        table = re.sub(r'\n\s*\n', '\n', table)       # 删除多余空行
        table = re.sub(r'\s*\n\s*', '\n', table)      # 去除行尾空格和多余换行
        
        cleaned_tables.append(table)
    
    return cleaned_tables

def process_tar_file(tar_path):
    # """从 .tar.gz 文件中提取 .tex 文件并处理其中的表格"""
    try:   # 有些压缩文件有错误，直接跳过，如压缩包中的文件仍然是压缩包
        with tarfile.open(tar_path, "r:gz") as tar:
            idx = 0  # 表格文件序号
            for member in tar.getmembers():
                if member.isfile() and member.name.endswith(".tex"):
                    tex_file = tar.extractfile(member)
                    if tex_file:
                        try:   # 处理无法用 UTF-8 编码解码的文件，程序会继续处理下一个文件，而不会中断
                            tex_content = tex_file.read().decode("utf-8")
                            tables = extract_tables(tex_content)
                            if not tables:
                                continue
                            # 获取文件基础名以便命名
                            base_id = os.path.basename(tar_path).replace("tar.gz", "")
                            # 将每个表格单独保存
                            for idx, table in enumerate(tables, start=idx):
                                output_file_path = os.path.join(output_dir, f"{base_id}_{idx}.tex")
                                with open(output_file_path, "w", encoding="utf-8") as f:
                                    f.write(table + "\n\n")
                            idx += 1

                        except UnicodeDecodeError as e:
                            print(f"Skipping {member.name} in {tar_path} due to decoding error: {e}")

    except (tarfile.ReadError, EOFError, OSError) as e:
        print(f"Failed to open {tar_path}: {e}")

# 遍历 source_dir 中的所有 .tar.gz 文件
for root, _, files in os.walk(source_dir):
    for file in files:
        if file.endswith(".tar.gz"):
            tar_path = os.path.join(root, file)
            print(f"Processing {tar_path}...")
            process_tar_file(tar_path)

print("表格提取完成！")


表格提取完成！
